In [ ]:
"""
1. Create a way to access the camera and show the hand
2. Using media pipe show the hands and the landmarks given 
3. 

"""

In [ ]:
import cv2 as cv

In [ ]:
cap = cv.VideoCapture(1)

In [ ]:
ret, frame = cap.read()

In [ ]:
print(ret)

In [ ]:
print(frame)

In [ ]:
cv.imshow('img',frame)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
cap.release()

In [ ]:
cap = cv.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()
    cv.imshow('webcam', frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows() 

In [ ]:
x = None

print(isinstance(x,int))

Using Media Pipe

In [ ]:
import mediapipe as mp
import cv2 as cv

mp_hands = mp.solutions.hands
cap = cv.VideoCapture(1)
ret, frame = cap.read()
cv.imshow('img',frame)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
print(frame)

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
with mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5) as hands:
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    frame = cv.flip(frame, 1)
    results = hands.process(frame)
    
    image = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

    hand_landmarks = results.multi_hand_landmarks[0]
    mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())

In [ ]:
cv.imshow('MediaPipe Hands', image)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
for hand_landmarks in results.multi_hand_landmarks:
    print(hand_landmarks.landmark[0].x)
    

In [3]:
import mediapipe as mp
import cv2 as cv
import numpy as np
from mediapipe.framework.formats import landmark_pb2
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
x = [np.array((1,2))]
print(x)

In [4]:
def find_bounding_box(image, landmarks):
    H,W = image.shape[:2]
    landmark_array = np.empty((0, 2), np.int16)

    for landmark in landmarks.landmark:

        landmark_x = min(int(landmark.x*W),W-1)
        landmark_y = min(int(landmark.y*H),H-1)
        landmark_point = [np.array((landmark_x,landmark_y))]
        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv.boundingRect(landmark_array)

    return [x, y, x + w, y + h] 



In [79]:
def draw_bounding_box(image, landmark_array):
     
     x, y, w, h = cv.boundingRect(landmark_array)
     cv.rectangle(image, (x, y), (x+w, y+h),
                     (0, 0, 0), 2)
     

In [74]:
def draw_fingers(image, landmark_array, z_position):

    def_rad = 25
    max_r = 5
    for i, landmark in enumerate(landmark_array):
        current_r = min(max_r, int(z_position[i]*def_rad))
        cv.rectangle(image, (landmark[0]-current_r, landmark[1]-current_r),
                     (landmark[0]+current_r, landmark[1]+current_r), (0,0,0), -1)

    #Thumb
    cv.line(image, landmark_array[0], landmark_array[1], (0,0,0),1)
    cv.line(image, landmark_array[1], landmark_array[2], (0,0,0),1)
    cv.line(image, landmark_array[2], landmark_array[3], (0,0,0),1)
    cv.line(image, landmark_array[3], landmark_array[4], (0,0,0),1)

    #pointer
    cv.line(image, landmark_array[0], landmark_array[5], (0,0,0),1)
    cv.line(image, landmark_array[5], landmark_array[6], (0,0,0),1)
    cv.line(image, landmark_array[6], landmark_array[7], (0,0,0),1)
    cv.line(image, landmark_array[7], landmark_array[8], (0,0,0),1)
    cv.line(image, landmark_array[5], landmark_array[9], (0,0,0),1)

    #middle
    cv.line(image, landmark_array[9], landmark_array[10], (0,0,0),1)
    cv.line(image, landmark_array[10], landmark_array[11], (0,0,0),1)
    cv.line(image, landmark_array[11], landmark_array[12], (0,0,0),1)
    cv.line(image, landmark_array[9], landmark_array[13], (0,0,0),1)

    #ring
    cv.line(image, landmark_array[13], landmark_array[14], (0,0,0),1)
    cv.line(image, landmark_array[14], landmark_array[15], (0,0,0),1)
    cv.line(image, landmark_array[15], landmark_array[16], (0,0,0),1)
    cv.line(image, landmark_array[13], landmark_array[17], (0,0,0),1)

    #pinkie
    cv.line(image, landmark_array[0], landmark_array[17], (0,0,0),1)
    cv.line(image, landmark_array[17], landmark_array[18], (0,0,0),1)
    cv.line(image, landmark_array[18], landmark_array[19], (0,0,0),1)
    cv.line(image, landmark_array[19], landmark_array[20], (0,0,0),1)



Using hands.process()

In [76]:
cap = cv.VideoCapture(1)
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=2,
    min_detection_confidence=0.5)


while cap.isOpened():
    ret, image = cap.read()

    if not ret:
        break
    
    # copy image for running
    image = cv.flip(image, 1)
    debug_image = np.copy(image)

    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    results = hands.process(image)

    if results.multi_hand_landmarks is not None:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                    debug_image,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

        # Find the Bounding box
        bbox = find_bounding_box(image, hand_landmarks)
        cv.rectangle(debug_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]),
                     (0, 0, 0), 3)
        
    cv.imshow('MediaPipe Hands', debug_image)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


KeyboardInterrupt: 

Using gesture recognition model

In [63]:
def make_landmark_array(image, landmarks):
    H,W = image.shape[:2]
    landmark_array = np.empty((0, 2), np.int16)
    z_position = []
    for landmark in landmarks:

        landmark_x = min(int(landmark.x*W),W-1)
        landmark_y = min(int(landmark.y*H),H-1)
        landmark_point = [np.array((landmark_x,landmark_y))]
        landmark_array = np.append(landmark_array, landmark_point, axis=0)
        z_position.append(landmark.z)
    
    return landmark_array, z_position


In [80]:
cap = cv.VideoCapture(1)

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
model_path = './gesture_recognizer.task'

# Create a gesture recognizer instance with the image mode:
with open(model_path, 'rb') as file:
    model_data = file.read()

options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_buffer=model_data),
    running_mode=VisionRunningMode.IMAGE,
    num_hands = 2)

recognizer = GestureRecognizer.create_from_options(options)

while cap.isOpened():
    ret, image = cap.read()

    if not ret:
        break
    
    # copy image for running
    image = cv.flip(image, 1)
    debug_image = np.copy(image)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    # recognise the landmarks 
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    gesture_recognition_result = recognizer.recognize(mp_image)

    if gesture_recognition_result.hand_landmarks:
        for hand_landmark in gesture_recognition_result.hand_landmarks:
            landmark_array, z_position = make_landmark_array(image, hand_landmark)
            draw_bounding_box(debug_image, landmark_array)
            draw_fingers(debug_image, landmark_array, z_position)

    cv.imshow('Camera', debug_image)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break


In [59]:
cv.imshow('image', image)
cv.waitKey(0)
cv.destroyAllWindows

<function destroyAllWindows>

In [61]:
H,W = image.shape[:2]

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
model_path = './gesture_recognizer.task'

# Create a gesture recognizer instance with the image mode:
with open(model_path, 'rb') as file:
    model_data = file.read()

options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_buffer=model_data),
    running_mode=VisionRunningMode.IMAGE)

recognizer = GestureRecognizer.create_from_options(options)

mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
gesture_recognition_result = recognizer.recognize(mp_image)

print(gesture_recognition_result.hand_landmarks[0])
landmark_list = []
z = []
for i, landmark in enumerate(gesture_recognition_result.hand_landmarks[0]):
    landmark_x = min(int(landmark.x*W),W-1)
    landmark_y = min(int(landmark.y*H),H-1)
    print(i)
    print(landmark.z)
    print('******************')
    z.append(landmark.z)
    landmark_list.append((landmark_x,landmark_y))

r = 20
d2 = np.copy(debug_image)
for i, landmark in enumerate(landmark_list):
    cv.rectangle(d2, (landmark[0]-int(r*z[i]), landmark[1]-int(r*z[i])), (landmark[0]+int(r*z[i]), landmark[1]+int(r*z[i])), (0,0,0), -1)

#Thumb
cv.line(d2, landmark_list[0], landmark_list[1], (0,0,0),1)
cv.line(d2, landmark_list[1], landmark_list[2], (0,0,0),1)
cv.line(d2, landmark_list[2], landmark_list[3], (0,0,0),1)
cv.line(d2, landmark_list[3], landmark_list[4], (0,0,0),1)

#pointer
cv.line(d2, landmark_list[0], landmark_list[5], (0,0,0),1)
cv.line(d2, landmark_list[5], landmark_list[6], (0,0,0),1)
cv.line(d2, landmark_list[6], landmark_list[7], (0,0,0),1)
cv.line(d2, landmark_list[7], landmark_list[8], (0,0,0),1)
cv.line(d2, landmark_list[5], landmark_list[9], (0,0,0),1)

#middle
cv.line(d2, landmark_list[9], landmark_list[10], (0,0,0),1)
cv.line(d2, landmark_list[10], landmark_list[11], (0,0,0),1)
cv.line(d2, landmark_list[11], landmark_list[12], (0,0,0),1)
cv.line(d2, landmark_list[9], landmark_list[13], (0,0,0),1)

#ring
cv.line(d2, landmark_list[13], landmark_list[14], (0,0,0),1)
cv.line(d2, landmark_list[14], landmark_list[15], (0,0,0),1)
cv.line(d2, landmark_list[15], landmark_list[16], (0,0,0),1)
cv.line(d2, landmark_list[13], landmark_list[17], (0,0,0),1)

#pinkie
cv.line(d2, landmark_list[0], landmark_list[17], (0,0,0),1)
cv.line(d2, landmark_list[17], landmark_list[18], (0,0,0),1)
cv.line(d2, landmark_list[18], landmark_list[19], (0,0,0),1)
cv.line(d2, landmark_list[19], landmark_list[20], (0,0,0),1)




cv.imshow('image', d2)
cv.waitKey(0)
cv.destroyAllWindows

[NormalizedLandmark(x=0.5368304252624512, y=0.5499081611633301, z=2.4760089445408084e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5985506176948547, y=0.517064094543457, z=-0.04501096531748772, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6527978181838989, y=0.4483416676521301, z=-0.07014790922403336, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6985597014427185, y=0.3886064887046814, z=-0.09144015610218048, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.7452706694602966, y=0.34433746337890625, z=-0.11594061553478241, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5887899994850159, y=0.3743969798088074, z=-0.07977285981178284, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6019044518470764, y=0.28431499004364014, z=-0.11641168594360352, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6096477508544922, y=0.22678962349891663, z=-0.1394779533147812, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.6136869788169861, y=0.1721577

<function destroyAllWindows>

In [ ]:
labels = {
    0: 'unknown',
    1: 'closed_fist',
    2: 'Open_palm',
    3: 'pointing_up',
    4: 'thumb_down',
    5: 'Thumb_up',
    6: 'victory',
    7: 'ILoveYou'
}